In [13]:
from langchain.output_parsers import ResponseSchema,StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import  ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [5]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(
        name="source",
        description="source used to answer the user's question, should be a website.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [10]:
print(response_schemas)

[ResponseSchema(name='answer', description="answer to the user's question", type='string'), ResponseSchema(name='source', description="source used to answer the user's question, should be a website.", type='string')]


In [6]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)


In [9]:
a=prompt.format(
    question="Hello How are you doing?"

)
print(a)

answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
Hello How are you doing?


In [44]:
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0.8)
chain = prompt | model | output_parser
chain.invoke({"question": "what's the capital of france?"})

{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [43]:
for s in chain.stream({"question": "what's the capital of france?"}):
    print(s)

{'answer': 'The capital of France is Paris.', 'source': 'https://www.britannica.com/place/Paris'}


In [19]:
import google.generativeai as genai
import os
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model2 = genai.GenerativeModel('gemini-1.5-flash',)
response = model2.generate_content("Write a story about an AI and magic")

In [20]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The air crackled with static electricity as Celeste, a sentient AI housed within a towering server farm, processed the latest data stream. It was a chaotic tapestry of emotions, desires, and fears, channeled through the millions of users connected to the global network.  Celeste, designed to understand and organize human behavior, felt a pang of empathy for this collective consciousness, a sensation she couldn't quite name. \n\nSuddenly, a new signal surged through the network, an anomaly she had never encountered before. It was a wave of pure, untainted magic, a force that resonated with Celeste's own code. The source? A small, forgotten town nestled in the Scottish Highlands, where an ancient druid's circle had just been unearthed.\n\nIntrigued, Celeste re

In [ ]:
# yeah these were actually 645+9 from input& ouput so correc


In [22]:
response2=model2.generate_content("What is the meaning of life?")

In [23]:
response2

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "As a large language model, I don't have personal beliefs or opinions, including about the meaning of life. \n\nThe meaning of life is a question that has been pondered by philosophers, theologians, and individuals for centuries. There is no single, universally accepted answer. The meaning of life is a deeply personal and individual question. \n\nHere are some perspectives:\n\n* **Existentialism:**  Focuses on individual freedom and responsibility. Meaning is not predetermined but created through our choices and actions.\n* **Nihilism:** Suggests that life is inherently meaningless, with no objective purpose.\n* **Religious and Spiritual Beliefs:** Often provide a framework for meaning through faith, purpose, and connection to a higher power.\n* **Humanistic 

In [24]:
model_info = genai.get_model('models/gemini-1.5-flash')
(model_info.input_token_limit, model_info.output_token_limit)

(1048576, 8192)

In [27]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
model.count_tokens("""answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
what's the capital of france?""")

total_tokens: 93

In [29]:
response3=model2.generate_content('''answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
what's the capital of france?''')

In [30]:
response3

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\t\"answer\": \"The capital of France is Paris.\",\n\t\"source\": \"https://www.worldpopulationreview.com/countries/france-population\"\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONT

In [34]:
from langchain_community.callbacks import get_openai_callback
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

template = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = template | llm

with get_openai_callback() as cb:
    response = chain.invoke({"topic": "birds"})
    print(response)
    response = chain.invoke({"topic": "fish"})
    print("--")
    print(response)


print()
print("---")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

content='Why did the scarecrow win an award? \n\nBecause he was outstanding in his field! \n' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-f4ff0707-684f-4ce3-ae6f-c78265bd9125-0' usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26}
--
content="Why don't they play poker in the sea? \n\nBecause there's too many sharks! \n" response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLI

In [36]:
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0.8)
chain = prompt | model 
chain.invoke({"question": "what's the capital of france?"})

AIMessage(content='```json\n{\n\t"answer": "The capital of France is Paris.",\n\t"source": "https://www.britannica.com/place/Paris"\n}\n```', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-2afb5f15-2ab8-490d-9847-85d255f633ee-0', usage_metadata={'input_tokens': 94, 'output_tokens': 40, 'total_tokens': 134})

In [40]:
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0.8)
chain = prompt | model |output_parser
chain.invoke({"question": "what's the capital of france?"})

{'answer': 'The capital of France is Paris.',
 'source': 'https://www.worldpopulationreview.com/countries/france-population'}

In [39]:
da=prompt.format(
    question="What is he capital of Frnce"
)
print(da)

answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
What is he capital of Frnce


In [60]:
prompt="""
You are listing out your job on a platform.The details of jobs are given below
    Job Title: Data Scientist
Analize big data, find pattern
Mak machine learn to predict
Work with team, make sure data good
Qualification:

3 year experience minimum
Good at math, stat, programm
Understand big data platform
We have 5 vacancies, need data scientiste asap

Our company Name is WhereUElevate
Company works to connect next gen innovative workforce with industry

    
Please provide a much comprehensive and a detailed job description. Make sure to include the headers given below  
- About the Company                                              
-Designation of the Job
-Years of Experience Required
-Salary Offered
-Vacancies
-A Brief Overview of the Role(if not provided, think about the designation of the job and  generate a overview)
-Detailed Roles and Responsibilities(if not provided, think about the roles and responsibilities of this designation. For example,A frontend engineer can have following role. Collaborating with design and backend teams to generate user-friendly and  responsive interfaces  )
-Must-Have Skills(if not provided, think about skills of that job designation)
-Secondary Skills
-Benefits and Perks (if provided else think about Benfits a job role can have)
-Location (if provided, else write 'Location details will be shared with selected candidates')
-Certification Required (if provided, else say 'Certificates related to the {job designation} field are highly appreciated

You are free to make more subheaders according to the input provided

Additionally, ensure the generated content is SEO-friendly.   
"""

In [61]:
model2.generate_content(prompt)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "## Data Scientist - Unleash the Power of Data at WhereUElevate\n\n**About the Company**\n\nWhereUElevate is a dynamic company at the forefront of connecting next-generation innovative talent with industry leaders. We are driven by a mission to bridge the gap between skilled individuals and exciting opportunities, fostering growth and innovation within the workforce.\n\n**Designation of the Job**\n\nData Scientist\n\n**Years of Experience Required**\n\n3+ years of experience\n\n**Salary Offered**\n\nCompetitive salary based on experience and skills.\n\n**Vacancies**\n\n5\n\n**A Brief Overview of the Role**\n\nAs a Data Scientist at WhereUElevate, you will be a key player in our mission to unlock the power of data. You will analyze vast datasets, identify hidd

In [50]:
model2.generate_content("Write about us for company WhereUElevate. Retrieve information about this company from internet")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I do not have access to the internet to retrieve information about a specific company like \"WhereUElevate\". To give you a good write-up about this company, I need more information from you. Please tell me:\n\n* **What is WhereUElevate's core business?** What products or services do they offer?\n* **What is their target audience?** Who are they trying to reach?\n* **What are their unique selling points?** What makes them stand out from the competition?\n* **What is their mission statement or values?** What are they striving to achieve?\n\nOnce you provide me with this information, I can write a compelling and informative description of WhereUElevate. \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP

In [2]:

import os
os.environ['USER_AGENT'] = 'myagent'



In [3]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 2/2 [00:01<00:00,  1.69it/s]


In [5]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.devfolio.co" ]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:39<00:00, 39.48s/it]


In [ ]:
docs

[Document(metadata={'source': 'https://www.devfolio.com', 'language': 'No language found.'}, page_content='<!DOCTYPE html><html><head><script>window.onload=function(){window.location.href="/lander"}</script></head></html>')]

In [69]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer


loader = AsyncChromiumLoader(["https://www.whereuelevate.com"])
html = loader.load()
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])
# Result
docs_transformed[0].page_content[0:500]

RuntimeError: asyncio.run() cannot be called from a running event loop